In [1]:
import requests
url = 'https://worldcup.sfg.io/matches'
r = requests.get(url)
all_games = r.json()

In [2]:
from datetime import datetime as dt
from sqlalchemy import Column, Integer, String, ForeignKey, DateTime, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, backref

Base = declarative_base()

class Game(Base):
    __tablename__ = 'games'
    id = Column(Integer, primary_key=True)
    venue = Column(String)
    winner = Column(String)
    statistics = relationship('Statistics', back_populates="game")
    teams = relationship(
        'Team',
        secondary='statistics',
        back_populates='games'
    )

class Team(Base):
    __tablename__ = 'teams'
    id = Column(Integer, primary_key=True)
    country = Column(String)
    statistics = relationship('Statistics', back_populates="team")
    games = relationship(
        'Game',
        secondary='statistics',
        back_populates='teams'
    )

class Statistics(Base):
    __tablename__ = 'statistics'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    game_id = Column(Integer, ForeignKey('games.id'))
    team_id = Column(Integer, ForeignKey('teams.id'))
    goals = Column(Integer)
    game = relationship('Game', back_populates='statistics')
    team = relationship('Team', back_populates='statistics')
    ball_possession=Column(Integer)
    distance_covered=Column(Integer)
    on_target=Column(Integer)
    pass_accuracy=Column(Integer)

engine = create_engine('sqlite:///worldcup.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
all_games_tuples= [(game['venue'], game['home_team_country'], game['away_team_country'], game['winner']) for game in all_games]

In [4]:
h_teams = set([t[1] for t in all_games_tuples])
a_teams = set([t[2] for t in all_games_tuples])
teams = h_teams.union(a_teams)

In [5]:
team_objects = [Team(country = t) for t in teams]
len(team_objects)

32

In [6]:
session.add_all(team_objects)
session.commit()

In [7]:
all_game_objects = []
all_stats = []
for game in all_games:
    h_team = session.query(Team).filter(Team.country == game['home_team_statistics']['country']).one()
    a_team = session.query(Team).filter(Team.country == game['away_team_statistics']['country']).one()
    game_object = Game(venue= game['venue'],
                       winner = game['winner'])
    all_game_objects.append(game_object)


    h_name = game['home_team']['country']
    a_name = game['away_team']['country']
    h_goals = game['home_team']['goals']
    h_ball_possession = game['home_team_statistics']['ball_possession']
    a_goals = game['away_team']['goals']
    a_ball_possession = game['away_team_statistics']['ball_possession']
    h_distance_covered = game['home_team_statistics']['distance_covered']
    a_distance_covered = game['away_team_statistics']['distance_covered']
    h_pass_accuracy = game['home_team_statistics']['pass_accuracy']
    a_pass_accuracy = game['away_team_statistics']['pass_accuracy']
    h_on_target = game['home_team_statistics']['on_target']
    a_on_target = game['away_team_statistics']['on_target']


    h_stats = Statistics(game = game_object,
                         name= h_name,
                         team = h_team,
                        ball_possession =h_ball_possession,
                        goals = h_goals,
                        distance_covered = h_distance_covered,
                        pass_accuracy = h_pass_accuracy,
                        on_target = h_on_target)
    a_stats = Statistics(game = game_object,
                         name= a_name,
                        team = a_team,
                        ball_possession =a_ball_possession,
                        goals = a_goals,
                        distance_covered = a_distance_covered,
                        pass_accuracy = a_pass_accuracy,
                        on_target = a_on_target)
    all_stats.append(h_stats)
    all_stats.append(a_stats)


In [8]:
len(all_game_objects)

64

In [9]:
len(all_stats)

128

In [10]:
session.add_all(all_game_objects)
session.add_all(all_stats)
session.commit()

In [11]:
#all statistics objects where russia played
session.query(Statistics).join(Team).filter(Team.country == 'Russia').all()

In [14]:
from sqlalchemy.sql import func

In [46]:
def top_eight_average_dc_per_game_desc():
    return session.query(Team.country, func.avg(Statistics.distance_covered)).group_by(Statistics.name).order_by(func.avg(Statistics.distance_covered).desc()).join(Statistics)[0:8]

top_eight_average_dc_per_game_desc()

[('Russia', 125.0),
 ('Croatia', 116.71428571428571),
 ('England', 116.42857142857143),
 ('Denmark', 115.5),
 ('Serbia', 112.0),
 ('Spain', 112.0),
 ('Germany', 111.33333333333333),
 ('Australia', 109.66666666666667)]

In [60]:
def return_venue_by_avg_goals_desc():
    return session.query(Game.venue, func.avg(Statistics.goals)).join(Statistics).group_by(Game.venue).order_by(Game.venue).all()

return_venue_by_avg_goals_desc()

[('Ekaterinburg', 1.125),
 ('Kaliningrad', 1.25),
 ('Kazan', 1.5833333333333333),
 ('Moscow', 1.4166666666666667),
 ('Nizhny Novgorod', 1.5833333333333333),
 ('Rostov-On-Don', 1.4),
 ('Samara', 0.9166666666666666),
 ('Saransk', 1.125),
 ('Sochi', 1.75),
 ('St. Petersburg', 1.0),
 ('Volgograd', 1.125)]

In [63]:
def return_venue_by_sum_goals_desc():
    return session.query(Game.venue, func.sum(Statistics.goals)).join(Statistics).group_by(Game.venue).order_by(Game.venue).all()

return_venue_by_sum_goals_desc()

[('Ekaterinburg', 9),
 ('Kaliningrad', 10),
 ('Kazan', 19),
 ('Moscow', 34),
 ('Nizhny Novgorod', 19),
 ('Rostov-On-Don', 14),
 ('Samara', 11),
 ('Saransk', 9),
 ('Sochi', 21),
 ('St. Petersburg', 14),
 ('Volgograd', 9)]

In [68]:

def return_game_winners_by_goals_desc():
    return session.query(Game.winner, func.count(Game.winner)).group_by(Game.winner).order_by(func.count(Game.winner).desc()).all()
return_game_winners_by_goals_desc()

[('Draw', 9),
 ('Belgium', 6),
 ('Croatia', 6),
 ('France', 6),
 ('England', 4),
 ('Uruguay', 4),
 ('Brazil', 3),
 ('Russia', 3),
 ('Sweden', 3),
 ('Colombia', 2),
 ('Mexico', 2),
 ('Argentina', 1),
 ('Denmark', 1),
 ('Germany', 1),
 ('Iran', 1),
 ('Japan', 1),
 ('Korea Republic', 1),
 ('Nigeria', 1),
 ('Peru', 1),
 ('Poland', 1),
 ('Portugal', 1),
 ('Saudi Arabia', 1),
 ('Senegal', 1),
 ('Serbia', 1),
 ('Spain', 1),
 ('Switzerland', 1),
 ('Tunisia', 1)]

In [70]:
def return_top_eight_teams_by_num_wins():
    return session.query(Game.winner, func.count(Game.winner)).group_by(Game.winner).order_by(func.count(Game.winner).desc())[1:9]
return_top_eight_teams_by_num_wins()

[('Belgium', 6),
 ('Croatia', 6),
 ('France', 6),
 ('England', 4),
 ('Uruguay', 4),
 ('Brazil', 3),
 ('Russia', 3),
 ('Sweden', 3)]